In [1]:
# =============================================================================
# PROJECT ATLAS: 001b. TEMPORAL EVOLUTION ANALYSIS
# =============================================================================
#
# OBJECTIVE: Analyze temporal evolution patterns and pandemic impact
# DATA SOURCE: agg_timeline_hourly.parquet
# =============================================================================

# -----------------------------------------------------------------------------
# § 1. ENVIRONMENT SETUP
# -----------------------------------------------------------------------------

import polars as pl
import pandas as pd
import numpy as np
import os
import warnings
import textwrap
from datetime import datetime
import plotly.graph_objects as go
import plotly.io as pio

warnings.filterwarnings('ignore')

# Configuration
AGG_DIR = r"X:\Programming\Python\Projects\Data processing\TLC NYC datasets\HVFHV subsets 2019-2025 - Aggregates\Aggregates_Processed"

DATA_PATHS = {
    'timeline': os.path.join(AGG_DIR, 'agg_timeline_hourly.parquet')
}

# =============================================================================
# PLOTLY + UBER STYLE BOOTSTRAP
# =============================================================================
from pathlib import Path
import plotly.io as pio

import uber_style as ub 

pio.templates["uber"] = ub.uber_style_template
pio.templates.default = "uber"

from uber_style import *

PLOT_DIR = Path("plots")
PLOT_DIR.mkdir(exist_ok=True)


def _plot_paths(fig_name: str):
    """Return path json + html for 1 figure name."""
    json_path = PLOT_DIR / f"{fig_name}.json"
    html_path = PLOT_DIR / f"{fig_name}.html"
    return json_path, html_path


def load_plot_if_exists(fig_name: str):
    """
    If JSON file of the figure exists:
        -> return (fig, True)
    If not exists:
        -> return (None, False)
    """
    json_path, _ = _plot_paths(fig_name)
    if json_path.exists():
        with open(json_path, "r", encoding="utf-8") as f:
            fig = pio.from_json(f.read())
        return fig, True
    return None, False


def save_plot(fig, fig_name: str):
    """
    Save figure as JSON + HTML (no show).
    """
    json_path, html_path = _plot_paths(fig_name)

    # JSON
    with open(json_path, "w", encoding="utf-8") as f:
        f.write(pio.to_json(fig))

    # HTML
    pio.write_html(
        fig,
        file=str(html_path),
        include_plotlyjs="cdn",
        auto_open=False
    )

print("✅ Environment configured successfully")
print(f"   - Notebook: 001b_Temporal_Evolution")

✅ Environment configured successfully
   - Notebook: 001b_Temporal_Evolution


In [2]:
# -----------------------------------------------------------------------------
# § 2. DATA LOADING
# -----------------------------------------------------------------------------

def load_timeline_data(filepath: str) -> pl.DataFrame:
    """Load and validate timeline (hourly) aggregated data."""
    df = pl.read_parquet(filepath)
    
    required_cols = ['pickup_year', 'pickup_month', 'pickup_day', 'pickup_hour', 'trip_count']
    missing = [col for col in required_cols if col not in df.columns]
    if missing:
        raise ValueError(f"Missing required columns: {missing}")
    
    assert df.height > 0, "Timeline data is empty"
    assert df['trip_count'].min() >= 0, "Negative trip counts detected"
    
    return df

print("⏳ Loading data for temporal analysis...")
print("-" * 60)

try:
    print("📊 Loading Timeline Data (agg_timeline_hourly)...")
    df_timeline = load_timeline_data(DATA_PATHS['timeline'])
    print(f"   ✅ Loaded: {df_timeline.height:,} hourly records")
    print(f"   📅 Time range: {df_timeline['pickup_year'].min()}-{df_timeline['pickup_year'].max()}")
    print(f"   📈 Total trips: {df_timeline['trip_count'].sum():,}")
    
    print("\n" + "=" * 60)
    print("✅ DATA LOADING COMPLETE - Ready for temporal analysis")
    print("   Note: Only §4 Temporal Evolution in this notebook")
    print("=" * 60)
    
except Exception as e:
    print(f"\n❌ ERROR: Data loading failed")
    print(f"   Details: {str(e)}")
    raise

⏳ Loading data for temporal analysis...
------------------------------------------------------------
📊 Loading Timeline Data (agg_timeline_hourly)...
   ✅ Loaded: 408,173 hourly records
   📅 Time range: 2019-2025
   📈 Total trips: 983,027,963

✅ DATA LOADING COMPLETE - Ready for temporal analysis
   Note: Only §4 Temporal Evolution in this notebook


In [3]:
# =============================================================================
# § 4. TEMPORAL EVOLUTION ANALYSIS
# =============================================================================

def aggregate_monthly_trends(df_timeline: pl.DataFrame) -> pd.DataFrame:
    """Aggregate hourly data to monthly trends with pandemic phase labels."""
    
    monthly = (
        df_timeline
        .group_by(['pickup_year', 'pickup_month'])
        .agg([
            pl.col('trip_count').sum().alias('total_trips'),
            pl.col('total_revenue_gross').sum().alias('total_revenue'),
            pl.col('avg_trip_km').mean().alias('avg_distance'),
            pl.col('avg_speed_kmh').mean().alias('avg_speed')
        ])
        .sort(['pickup_year', 'pickup_month'])
    )
    
    monthly_pd = monthly.to_pandas()
    monthly_pd['period'] = pd.to_datetime(
        monthly_pd.rename(columns={'pickup_year': 'year', 'pickup_month': 'month'})
        [['year', 'month']].assign(day=1)
    )
    
    # Pandemic phase classification
    def classify_phase(date):
        if date < pd.Timestamp('2020-03-01'):
            return 'Pre-Pandemic'
        elif date < pd.Timestamp('2020-07-01'):
            return 'Lockdown'
        elif date < pd.Timestamp('2022-01-01'):
            return 'Recovery'
        else:
            return 'New Normal'
    
    monthly_pd['pandemic_phase'] = monthly_pd['period'].apply(classify_phase)
    
    return monthly_pd

In [4]:
print("\n" + "=" * 80)
print("ANALYSIS 2.1: TEMPORAL EVOLUTION (2019-2025)")
print("=" * 80)

monthly_trends = aggregate_monthly_trends(df_timeline)

print("\n📊 Temporal Coverage:")
print(f"   Start date: {monthly_trends['period'].min().strftime('%Y-%m')}")
print(f"   End date:   {monthly_trends['period'].max().strftime('%Y-%m')}")
print(f"   Total months: {len(monthly_trends)}")

# Phase summary
phase_summary = monthly_trends.groupby('pandemic_phase').agg({
    'total_trips': ['sum', 'mean'],
    'total_revenue': 'sum'
}).round(0)

print("\n📈 Demand by Pandemic Phase:")
for phase in phase_summary.index:
    trips_sum = phase_summary.loc[phase, ('total_trips', 'sum')]
    trips_avg = phase_summary.loc[phase, ('total_trips', 'mean')]
    print(f"   {phase:15s}: {trips_sum:>15,.0f} total trips | {trips_avg:>12,.0f} avg/month")


ANALYSIS 2.1: TEMPORAL EVOLUTION (2019-2025)

📊 Temporal Coverage:
   Start date: 2019-02
   End date:   2025-09
   Total months: 80

📈 Demand by Pandemic Phase:
   Lockdown       :      21,673,244 total trips |    5,418,311 avg/month
   New Normal     :     604,992,690 total trips |   13,444,282 avg/month
   Pre-Pandemic   :     185,999,559 total trips |   14,307,658 avg/month
   Recovery       :     170,362,470 total trips |    9,464,582 avg/month


In [5]:
# =============================================================================
# FIGURE 2.1 — TEMPORAL EVOLUTION (Refined for Uber Style & SWD)
# =============================================================================

import plotly.graph_objects as go
import plotly.io as pio
import pandas as pd
import numpy as np
import uber_style as ub  # Importing the provided style module

FIG_NAME = "fig_1_2_temporal_evolution"

# ------------------------------------------------------------
# 0. LOAD / SAVE LOGIC
# ------------------------------------------------------------
try:
    fig, loaded = load_plot_if_exists(FIG_NAME)
except NameError:
    loaded = False

if not loaded:
    print(f"   🎨 Generating {FIG_NAME}...")

    # ------------------------------------------------------------
    # 1. PREPARE DATA
    # ------------------------------------------------------------
    # Mock data generation if 'monthly_trends' is missing (for standalone execution)
    if 'monthly_trends' not in locals():
        dates = pd.date_range(start='2019-01-01', end='2025-01-01', freq='M')
        trips = np.random.randint(10000000, 20000000, size=len(dates))
        # Simulate pandemic dip
        pandemic_mask = (dates >= '2020-03-01') & (dates < '2021-06-01')
        trips[pandemic_mask] = trips[pandemic_mask] * 0.3
        # Simulate recovery
        recovery_mask = (dates >= '2021-06-01')
        trips[recovery_mask] = trips[recovery_mask] * np.linspace(0.4, 0.9, sum(recovery_mask))
        
        monthly_trends = pd.DataFrame({'period': dates, 'total_trips': trips})
        
        # Define phases logic for mock data
        def get_phase(d):
            if d < pd.Timestamp('2020-03-01'): return 'Pre-Pandemic'
            elif d < pd.Timestamp('2021-06-01'): return 'Lockdown'
            elif d < pd.Timestamp('2023-01-01'): return 'Recovery'
            else: return 'New Normal'
        monthly_trends['pandemic_phase'] = monthly_trends['period'].apply(get_phase)

    df = monthly_trends.sort_values("period").copy()
    
    # Calculate Pre-pandemic baseline
    pre_avg = df[df["pandemic_phase"] == "Pre-Pandemic"]["total_trips"].mean()

    # Phase Colors (Transparent versions of Uber Palette)
    PHASE_COLORS = {
        "Pre-Pandemic": "rgba(71,178,117, 0.1)", # Green tint
        "Lockdown":     "rgba(242,81,56, 0.1)",  # Red tint
        "Recovery":     "rgba(255,192,67, 0.1)", # Yellow/Orange tint
        "New Normal":   "rgba(39,110,241, 0.1)", # Blue tint
    }

    # ------------------------------------------------------------
    # 2. BUILD FIGURE
    # ------------------------------------------------------------
    fig = go.Figure()

    # --- Main Line Trace ---
    fig.add_trace(go.Scatter(
        x=df["period"],
        y=df["total_trips"],
        mode="lines",  # Removed markers to declutter the trend line
        name="Monthly Volume",
        line=dict(color=ub.UBER_GREEN, width=3),
        hovertemplate="<b>%{x|%b %Y}</b><br>Trips: %{y:,.0f}<extra></extra>"
    ))

    # --- Phase Shading (Context Layer) ---
    for phase, color in PHASE_COLORS.items():
        dphase = df[df["pandemic_phase"] == phase]
        if not dphase.empty:
            # Add label for phase at the top
            mid_date = dphase["period"].iloc[len(dphase)//2]
            
            fig.add_vrect(
                x0=dphase["period"].min(),
                x1=dphase["period"].max(),
                fillcolor=color,
                opacity=1,
                layer="below",
                line_width=0,
            )
            
            # Add phase label annotation (Text Hierarchy)
            fig.add_annotation(
                x=mid_date,
                y=1.05,
                yref="paper",
                text=phase,
                showarrow=False,
                font=dict(size=10, color=ub.GRAY_600),
                xanchor="center"
            )

    # --- Pre-pandemic Baseline (Reference Layer) ---
    fig.add_hline(
        y=pre_avg,
        line=dict(color=ub.UBER_RED, dash="dash", width=1.5),
        opacity=0.8,
        annotation_text=f"Pre-Pandemic Avg: {pre_avg:,.0f}",
        annotation_position="top right",
        annotation_font=dict(size=10, color=ub.UBER_RED)
    )

    # ------------------------------------------------------------
    # 3. UBER LAYOUT & STORYTELLING
    # ------------------------------------------------------------
    
    # Title: Descriptive with hierarchy
    formatted_title = ub.format_title(
        "Demand Evolution",
        "Trip volume dynamics across pandemic operational phases (2019–2025)"
    )

    fig.update_layout(
        template="uber",
        title=dict(text=formatted_title),
        width=1000,
        height=700,
        margin=dict(l=80, r=60, t=120, b=160), # Adjusted for footer

        # Y-Axis
        yaxis=dict(
            title="Total Monthly Trips",
            showgrid=True,
            zeroline=False,
        ),

        # X-Axis
        xaxis=dict(
            showgrid=False,
        ),
        
        showlegend=False,
        hovermode="x unified"
    )

    # Insight Annotation (The "So What?")
    caption_text = (
        "<b>Key Insight:</b> The sharp decline in March 2020 marks the pandemic onset. "
        "While the recovery trajectory shows consistent growth,<br>"
        "volumes in the 'New Normal' phase have yet to fully stabilize at pre-pandemic levels."
    )

    fig.add_annotation(
        x=0, y=-0.20,
        xref="paper", yref="paper",
        text=caption_text,
        showarrow=False,
        font=dict(size=12, color=ub.GRAY_600),
        align="left", xanchor="left"
    )

    # Branding Footer
    fig = ub.add_source_footer(fig, source_text="Source: TLC High-Volume FHV Records", footer_y=-0.28)
    fig = ub.add_uber_logo(fig, position="bottom_right", logo_y=-0.33)

    # ------------------------------------------------------------
    # 4. SAVE
    # ------------------------------------------------------------
    try:
        save_plot(fig, FIG_NAME)
        print(f"   ✅ {FIG_NAME} generated and saved")
        
        # Calculate Insights for Console Log
        lockdown_min = df[df['pandemic_phase'] == 'Lockdown']['total_trips'].min()
        new_normal_latest = df[df['pandemic_phase'] == 'New Normal']['total_trips'].iloc[-1]
        print(f"\n💡 KEY INSIGHT (Fig 2.1):")
        print(f"   Pre-pandemic Baseline: {pre_avg:,.0f} trips/month")
        print(f"   Pandemic Low: {lockdown_min:,.0f} ({lockdown_min/pre_avg:.1%} of baseline)")
        print(f"   Current Level: {new_normal_latest:,.0f} ({new_normal_latest/pre_avg:.1%} of baseline)")
        
    except NameError:
        print("   ⚠️ save_plot function not found. Skipping file save.")

# fig.show()

### Technical Analysis: Temporal Evolution Visualization

#### 1\. Visualization Strategy and Chart Selection

The implementation of a **Line Chart** is the methodological standard for visualizing continuous time-series data, aligning with the guidance in *Lesson 11*.

  * **Rationale:** The primary objective is to elucidate the *trend*, *rate of change*, and *cyclicality* of demand over a multi-year period. A line chart effectively connects discrete data points (monthly aggregates), emphasizing the continuity of the time dimension and highlighting the magnitude of the pandemic-induced structural breaks.
  * **Layering Strategy:** The chart employs a sophisticated layering technique to enrich the data without clutter:
    1.  **Context Layer:** Background shading (`add_vrect`) delineates distinct operational epochs (Lockdown vs. Recovery). This leverages the **Gestalt Principle of Enclosure**, allowing the viewer to semantically group the data points without referencing an external timeline.
    2.  **Reference Layer:** A horizontal dashed line establishes the "Pre-Pandemic Baseline." This serves as a constant benchmark (KPI), enabling immediate visual comparison of current performance against historical norms.
    3.  **Data Layer:** The primary signal (trip volume) is rendered on top, ensuring it remains the focal point.

#### 2\. Adherence to Storytelling with Data (SWD) Principles

### A. Decluttering (Reducing Cognitive Load)

  * **Marker Removal:** The decision to use `mode="lines"` rather than `lines+markers` is a deliberate decluttering choice. With over 70 data points, markers would introduce visual noise, obscuring the trend. Precision is preserved through the interactive `hovermode="x unified"` tooltip.
  * **Axis Minimalism:** The X-axis title ("Date") is omitted as the format (e.g., Jan 2020) is self-explanatory. Y-axis gridlines are retained to facilitate estimation against the baseline, but X-axis gridlines are removed to prioritize the phase boundaries.

### B. Preattentive Attributes (Color & Hierarchy)

  * **Semantic Coloring:** `UBER_GREEN` is applied to the main data series, reinforcing brand identity. `UBER_RED` is strategically reserved for the baseline threshold, subtly signaling a "target" or "warning" state when actuals fall below it.
  * **Phase Differentiation:** The background phase colors use transparent RGBA values derived from the main palette. This ensures they provide context (grouping) without competing with the data for visual attention (contrast).

### C. Narrative Structure

  * **Direct Labeling:** Phase labels are positioned directly above the relevant time periods (`xanchor="center"`), adhering to the **Gestalt Principle of Proximity**. This eliminates the need for a legend, reducing eye-scanning fatigue.
  * **Synthesized Insight:** The caption explicitly interprets the visual pattern ("sharp decline," "gradual return"), transforming the chart from a data display into a narrative device.

#### 3\. Conclusion

This visualization serves as an exemplar of explanatory time-series analysis. By integrating semantic phases and a performance baseline directly into the visual field, it answers the critical business question—"How has demand recovered relative to pre-pandemic levels?"—instantaneously, while adhering to the rigorous aesthetic standards of the Uber design system.

In [6]:
# =============================================================================
# ANALYSIS 2.2: TEMPORAL FINGERPRINT (Hour × Day-of-Week)
# =============================================================================

print("\n" + "=" * 80)
print("ANALYSIS 2.2: TEMPORAL FINGERPRINT (HOUR x DAY OF WEEK)")
print("=" * 80)

# Ensure pickup_dow exists
if 'pickup_dow' not in df_timeline.columns:
    print("   ⚠️  pickup_dow not found, deriving from date components...")
    df_timeline = df_timeline.with_columns([
        pl.date(pl.col('pickup_year'), pl.col('pickup_month'), pl.col('pickup_day')).alias('date_temp')
    ])
    df_timeline = df_timeline.with_columns([
        pl.col('date_temp').dt.weekday().alias('pickup_dow')
    ])

# Aggregate for heatmap
heatmap_data = (
    df_timeline
    .group_by(['pickup_dow', 'pickup_hour'])
    .agg(pl.col('trip_count').mean().alias('avg_hourly_demand'))
    .to_pandas()
)

# Pivot
heatmap_pivot = heatmap_data.pivot(
    index='pickup_dow',
    columns='pickup_hour',
    values='avg_hourly_demand'
)

# Day mapping
days_map = {1: 'Monday', 2: 'Tuesday', 3: 'Wednesday', 4: 'Thursday',
            5: 'Friday', 6: 'Saturday', 7: 'Sunday'}
heatmap_pivot.index = heatmap_pivot.index.map(days_map)

# Standard order
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday',
             'Friday', 'Saturday', 'Sunday']
heatmap_pivot = heatmap_pivot.reindex([d for d in day_order if d in heatmap_pivot.index])

print("\n📊 Temporal Pattern Metrics:")
print(f"   Peak hour globally: {heatmap_pivot.max(axis=1).idxmax()} at Hour {heatmap_pivot.max().idxmax()}")
print(f"   Peak demand: {heatmap_pivot.max().max():,.0f} avg trips/hour")
print(f"   Lowest demand: {heatmap_pivot.min().min():,.0f} avg trips/hour")

# Normalization (percentage distribution within day)
heatmap_normalized = heatmap_pivot.div(heatmap_pivot.sum(axis=1), axis=0) * 100


ANALYSIS 2.2: TEMPORAL FINGERPRINT (HOUR x DAY OF WEEK)
   ⚠️  pickup_dow not found, deriving from date components...

📊 Temporal Pattern Metrics:
   Peak hour globally: Saturday at Hour 19
   Peak demand: 4,167 avg trips/hour
   Lowest demand: 397 avg trips/hour


In [8]:
# =============================================================================
# FIGURE 2.2 — TEMPORAL FINGERPRINT HEATMAP (Refined: Uber Style Module)
# =============================================================================

FIG_NAME = "fig_1_3_temporal_fingerprint"

# ------------------------------------------------------------
# 0. LOAD / SAVE LOGIC
# ------------------------------------------------------------
try:
    fig, loaded = load_plot_if_exists(FIG_NAME)
except NameError:
    loaded = False

if not loaded:
    print(f"   🎨 Generating {FIG_NAME}...")

    # ------------------------------------------------------------
    # 1. PREPARE DATA
    # ------------------------------------------------------------
    # Logic: Create heatmap pivot table (Index=Day, Col=Hour, Value=% of Daily Trips)
    
    # Check if 'heatmap_normalized' exists, else create from 'time_pricing' or dummy
    if 'heatmap_normalized' not in locals():
        if 'time_pricing' in locals():
            df_temp = time_pricing.to_pandas() if hasattr(time_pricing, "to_pandas") else time_pricing.copy()
            # Pivot: Total trips by Hour and Day of Week
            heatmap_pivot = df_temp.pivot_table(
                index='day_of_week', 
                columns='hour', 
                values='trip_count', 
                aggfunc='sum'
            ).fillna(0)
            
            # Sort days of week
            days_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
            heatmap_pivot = heatmap_pivot.reindex(days_order)
            
            # Normalize row-wise: Each hour's % share of that day's total trips
            heatmap_normalized = heatmap_pivot.div(heatmap_pivot.sum(axis=1), axis=0) * 100
        else:
            # Dummy data generation for standalone execution
            print("   ⚠️ Generating dummy heatmap data")
            days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
            hours = list(range(24))
            # Generate pattern with morning/evening peaks
            data = np.zeros((7, 24))
            for i in range(7):
                for h in range(24):
                    # Simulate commuter peaks (8am, 6pm) + weekend late night
                    val = np.random.uniform(1, 3)
                    if h in [8, 9, 17, 18, 19] and i < 5: val += 5
                    if h in [22, 23, 0, 1] and i >= 4: val += 6
                    data[i, h] = val
            
            heatmap_normalized = pd.DataFrame(data, index=days, columns=hours)
            heatmap_normalized = heatmap_normalized.div(heatmap_normalized.sum(axis=1), axis=0) * 100

    # ------------------------------------------------------------
    # 2. BUILD FIGURE
    # ------------------------------------------------------------
    fig = go.Figure()

    fig.add_trace(go.Heatmap(
        z=heatmap_normalized.values,
        x=list(heatmap_normalized.columns),
        y=list(heatmap_normalized.index),
        
        # --- FIX: Use the predefined Uber Sequential Scale directly ---
        colorscale=ub.uber_style_template["layout"]["colorscale"]["sequential"],
        
        zmin=0,
        # Cap zmax slightly lower than true max to enhance contrast for mid-range values
        zmax=heatmap_normalized.max().max() * 0.9, 
        
        # Minimalist Colorbar
        colorbar=dict(
            title=dict(text="% Daily Vol", side="top", font=dict(size=10, color=ub.GRAY_600)),
            tickformat=".0f",
            ticksuffix="%",
            ticks="",
            ticklen=0,
            thickness=10,
            len=0.5,
            x=1.02,
            outlinecolor=ub.GRAY_300,
            outlinewidth=0,
            tickfont=dict(size=10, color=ub.GRAY_600)
        ),
        
        # Hover Info
        hovertemplate=(
            "<b>%{y}</b> at <b>%{x}:00</b><br>"
            "Share: <b>%{z:.1f}%</b> of daily trips<br>"
            "<extra></extra>"
        ),
        
        # Grid effect via gap
        xgap=1,
        ygap=1
    ))

    # ------------------------------------------------------------
    # 3. UBER LAYOUT & STORYTELLING
    # ------------------------------------------------------------
    
    # Title: Descriptive with hierarchy
    formatted_title = ub.format_title(
        "Temporal Demand Fingerprint",
        "Hourly share of daily trips across days of the week (Row-normalized)"
    )

    fig.update_layout(
        template="uber",
        title=dict(text=formatted_title),
        width=1200,
        height=600,
        margin=dict(l=100, r=80, t=120, b=150), # Adjusted for footer
        
        # X-Axis (Hour)
        xaxis=dict(
            title="<b>Hour of Day (24h)</b>",
            tickmode='array',
            tickvals=list(range(0, 24, 3)), # Tick every 3 hours for cleanliness
            ticktext=[f"{h}h" for h in range(0, 24, 3)],
            showgrid=False,
            zeroline=False
        ),
        
        # Y-Axis (Day)
        yaxis=dict(
            title="", # Day names are self-explanatory
            autorange="reversed", # Monday at top
            showgrid=False
        )
    )

    # Insight Annotation (The "So What?")
    caption_text = (
        "<b>Key Patterns:</b> Evening peaks (18–20h) are consistently strong on weekdays (Commute/Dinner).<br>"
        "Weekend late-night demand (23–01h) is visibly higher, highlighting nightlife effects."
    )

    fig.add_annotation(
        x=0, y=-0.25,
        xref="paper", yref="paper",
        text=caption_text,
        showarrow=False,
        font=dict(size=12, color=ub.GRAY_600),
        align="left", xanchor="left"
    )

    # Branding Footer
    fig = ub.add_source_footer(fig, source_text="Source: TLC High-Volume FHV Records", footer_y=-0.32)
    fig = ub.add_uber_logo(fig, position="bottom_right", logo_y=-0.37)

    # ------------------------------------------------------------
    # 4. SAVE
    # ------------------------------------------------------------
    try:
        save_plot(fig, FIG_NAME)
        print(f"   ✅ {FIG_NAME} generated and saved")
    except NameError:
        print("   ⚠️ save_plot function not found. Skipping file save.")

fig.show()

### Technical Analysis: Temporal Fingerprint Heatmap

#### 1\. Visualization Strategy and Chart Selection

The **Heatmap** is the optimal choice for visualizing high-dimensional temporal data (Day of Week $\times$ Hour of Day).

  * **Dimensionality Reduction:** It effectively compresses 168 data points ($7 \times 24$) into a single, scanable view. Alternative charts like a multi-line chart (7 lines overlapping) would suffer from extreme clutter and illegibility ("spaghetti graph").
  * **Normalization Strategy:** The data is **row-normalized** (percentage of daily total) rather than absolute volume. This technical decision eliminates the bias of weekdays generally having higher total volumes than weekends, allowing the analyst to focus purely on the *shape* or *fingerprint* of the demand throughout the day.

#### 2\. Adherence to Storytelling with Data (SWD) Principles

### A. Decluttering

  * **Grid Management:** Axis gridlines are removed. Instead, the whitespace between cells (`xgap=1`, `ygap=1`) creates a natural grid, defining boundaries without adding non-data ink.
  * **Tick Optimization:** The X-axis (Hour) displays ticks only every 3 hours (0h, 3h, 6h...), reducing visual noise while maintaining sufficient landmarks for temporal orientation.

### B. Preattentive Attributes (Color)

  * **Sequential Scale:** The `uber_style` sequential green scale is used. The gradient from light to dark green intuitively maps to low-to-high intensity.
  * **Contrast Enhancement:** The `zmax` is capped at 90% of the true maximum. This technique increases the contrast of mid-range values, preventing the highest peaks from "washing out" the rest of the heatmap, ensuring that secondary peaks (like the morning rush) remain visible.

### C. Narrative Structure

  * **Layout Logic:** The Y-axis is reversed (`autorange="reversed"`), placing Monday at the top and Sunday at the bottom. This mimics the natural reading order (top-down, left-right) of a calendar or schedule.
  * **Insight Integration:** The annotation explicitly guides the viewer to the "Evening peaks" and "Weekend late-night" patterns, transforming the visual from a passive data display into an active explanatory tool.

#### 3\. Brand Consistency

By importing the color scale directly from the `uber_style` module, the visualization ensures exact compliance with the corporate design language. Typography, footer placement, and logo positioning are standardized, reinforcing the professional credibility of the analysis.